In [3]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,fbeta_score
import re
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
dataset_path='Dataset\\'
train_dir=dataset_path+'data_train'
test_dir=dataset_path+'data_test'
batch_size=32
num_classes=2
input_shape=(64,64,3)
class_mode='categorical'

In [5]:
train_gen=ImageDataGenerator(
rescale=1./255,
rotation_range=60,
width_shift_range=0.3,
height_shift_range=0.3,
shear_range=0.3,
zoom_range=0.3,
horizontal_flip=True,
)

train_generator=train_gen.flow_from_directory(
directory=train_dir,
target_size=(64,64),
batch_size=batch_size,
class_mode=class_mode,
color_mode='rgb',
shuffle=True,
seed=69
)
test_gen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_gen.flow_from_directory(
    directory=test_dir,
    target_size=(64, 64),
    batch_size=1,
    class_mode=class_mode,
    color_mode='rgb',
    shuffle=False,
    seed=69
)


Found 3850 images belonging to 2 classes.
Found 1650 images belonging to 2 classes.


In [6]:
conv_base=VGG16(include_top=False,
               weights='imagenet',
               input_shape=input_shape,
               pooling='avg')
top_model=conv_base.output
output_layer=Dense(2,activation='softmax')(top_model)

model=Model(inputs=conv_base.input, outputs=output_layer)
for layer in conv_base.layers:
    layer.trainable=False

model.compile(optimizer='adam', loss='binary_crossentropy',
                 metrics=['accuracy'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [29]:
n_steps=train_generator.samples//batch_size
n_val_steps=test_generator.samples//batch_size
n_epochs=2

checkpoint=ModelCheckpoint(filepath='model.weights.best4.hdf5',
                          monitor='val_accuracy',
                          save_best_only=True,
                          verbose=1)
early_stop=EarlyStopping(monitor='val_accuracy',
                        restore_best_weights=True,
                        mode='max')

In [30]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=n_steps,
                             epochs=n_epochs,
                             callbacks=[early_stop, checkpoint],
                             validation_data=test_generator,
                             validation_steps=n_val_steps)

Epoch 1/2
120/120 [==============================] - 111s 921ms/step - loss: 0.4355 - accuracy: 0.8287 - val_loss: 0.2366 - val_accuracy: 0.9412

Epoch 00001: val_accuracy improved from -inf to 0.94118, saving model to model.weights.best4.hdf5
Epoch 2/2
120/120 [==============================] - 110s 920ms/step - loss: 0.2650 - accuracy: 0.9002 - val_loss: 0.0332 - val_accuracy: 0.9608

Epoch 00002: val_accuracy improved from 0.94118 to 0.96078, saving model to model.weights.best4.hdf5


In [7]:
def display_results(y_true,y_preds,class_labels):
    results=pd.DataFrame(precision_recall_fscore_support(y_true,y_preds),
                        columns=class_labels).T
    results.rename(columns={0: 'Precision',
                           1: 'Recall',
                           2: 'F-Score',
                           3: 'Support'}, inplace=True)
    
    conf_mat = pd.DataFrame(confusion_matrix(y_true, y_preds), 
                            columns=class_labels,
                            index=class_labels)    
    f2 = fbeta_score(y_true, y_preds, beta=2, average='micro')
    print(f"Global F2 Score: {f2}")    
    return results, conf_mat

In [8]:
model.load_weights('model.weights.best4.hdf5')

In [10]:
class_indices=train_generator.class_indices
class_indices=dict((v,k) for k,v in class_indices.items())
test_generator.reset()

predictions=model.predict_generator(test_generator,steps=len(test_generator.filenames))

In [12]:
len(predictions)

1650

In [13]:
predicted_classes=np.argmax(np.rint(predictions),axis=1)
true_classes=test_generator.classes

In [14]:
prf,cm=display_results(true_classes,predicted_classes,class_indices.values())

Global F2 Score: 0.9472727272727273


In [15]:
cm

,Industrial,Residential
Industrial,659,72
Residential,15,904


In [16]:
prf

,Precision,Recall,F-Score,Support
Industrial,0.977745,0.901505,0.938078,731.0
Residential,0.926230,0.983678,0.954090,919.0
